In [ ]:
import os 
import tempfile
import pandas as pd

In [ ]:
data_root = "/path/to/processed-surf-dataset/"
time_granularities = ['60s', '300s', '600s']

datasets = ['node_load1', 'node_sockstat_sockets_used']
dfs = []

def get_sizes_of_files(full_path):
    size = 0
    for path, dirs, files in os.walk(full_path):
        for f in files:
            fp = os.path.join(path, f)
            size += os.stat(fp).st_size
    return size

dataset_sizes = dict()

for dataset in datasets:
    dataset_sizes.setdefault('15s', 0)
    dataset_sizes['15s'] += get_sizes_of_files(os.path.join(data_root, dataset))
    df = pd.read_parquet(os.path.join(data_root, dataset), engine='pyarrow')
    df.index = pd.to_datetime(df.index, utc=True, unit="s")
    dfs.append(df)

for time_granularity in time_granularities:
    dataset_sizes.setdefault(time_granularity, 0)
    for df in dfs:
        with tempfile.NamedTemporaryFile() as fp:        
            temp = df.resample(time_granularity).mean()
            temp.to_parquet(fp, engine='pyarrow')
            dataset_sizes[time_granularity] += os.stat(fp.name).st_size

In [ ]:
for g, v in dataset_sizes.items():
    print(g, f"{v/1024/2014:,.2f}")